# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [4]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


In [5]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,santa maria del oro,25.9333,-105.3667,51.28,39,35,9.57,MX,1674009019
1,1,ushuaia,-54.8000,-68.3000,56.86,62,20,6.91,AR,1674008879
2,2,hermanus,-34.4187,19.2345,72.81,89,98,8.72,ZA,1674009020
3,3,lebu,-37.6167,-73.6500,54.99,92,10,9.89,CL,1674009020
4,4,cherskiy,68.7500,161.3000,-48.68,99,68,4.18,RU,1674009021


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [13]:
%%capture --no-display

# Configure the map plot
gmaps.configure(api_key=g_key)

# Display the map
locations = city_data_df[['Lat', 'Lng']]
humidity = city_data_df["Humidity"].astype(float)
layout = {'width': '1000px', 'height': '600px'}
fig = gmaps.figure(map_type = 'HYBRID', layout= layout)
heat_layer = gmaps.heatmap_layer(locations, weights= humidity, max_intensity= max(city_data_df['Humidity']), point_radius= 2)
fig.add_layer(heat_layer)
fig

AttributeError: module 'gmaps' has no attribute 'configure'

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [15]:
# Drop any rows with null values

perfect_weather_cities = city_data_df.loc[(city_data_df['Max Temp'] > 70) & (city_data_df['Max Temp'] < 80 )
                                      & (city_data_df['Humidity'] < 40) & (city_data_df['Cloudiness'] == 0)
                                                  & (city_data_df['Wind Speed'] < 10)]
perfect_weather_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date


### Step 3: Create a new DataFrame called `hotel_df`.

In [12]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = perfect_weather_df
hotel_df.reset_index(drop= True, inplace= True)

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [16]:
# Set parameters to search for a hotel
hotel_df = perfect_weather_cities

params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key }


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lon"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["location"] = f"{lat},{lng}"
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    response = requests.get(base_url, params=params)
    response = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lon"]]

In [17]:
hotel_layer = gmaps.symbol_layer(locations, info_box_content=hotel_info)
markers = gmaps.marker_layer(locations)
# Add the layers to the map
fig.add_layer(hotel_layer)
fig.add_layer(markers)
fig

AttributeError: module 'gmaps' has no attribute 'symbol_layer'